In [1]:
# recent gameweek
latest_gameweek = 17

In [2]:
import pandas as pd
import numpy as np

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + '.csv'
players = pd.read_csv(filepath, index_col=0)

In [4]:
# how many gameweeks do we look form for
for i in [5,10]:
    number_gameweeks = i
    # create a list of relevant column names
    column_list = []
    for i in range(0,number_gameweeks):
        column_list.append('xPoints week ' + str(latest_gameweek-i))
    # calculate form
    column_name = 'form ' + str(number_gameweeks)
    players[column_name] = players[column_list].mean(axis=1)

In [5]:
# give a sorted list showing the players with best 'form 5'
players[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game',\
                                'form 5', 'form 10']].sort_values(by='form 5', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game,form 5,form 10
id,,,,,,,,
302,Fernandes,16.049383,130,8.1,114.774728,7.151348,8.566375,8.427104
272,De Bruyne,13.965517,81,5.8,99.833152,7.148547,7.596205,8.216120
45,El Ghazi,9.038462,47,5.2,42.235904,4.672909,6.694115,4.752286
4,Aubameyang,15.142857,53,3.5,61.995316,4.094030,6.417507,4.357450
39,Hause,4.000000,28,7.0,21.549346,5.387336,6.371007,6.371007
271,Gündogan,10.000000,45,4.5,39.575574,3.957557,6.303573,4.907134
255,Robertson,17.021277,80,4.7,79.233667,4.654978,6.286418,5.279439
246,Matip,8.000000,40,5.0,34.403122,4.300390,6.223573,4.928497
572,Adarabioyo,10.857143,38,3.5,37.420871,3.446659,6.183487,4.044250


In [6]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + '.csv'
players.to_csv(filepath)